In [1]:
!pip install --upgrade pip
!pip install jax flax optax tensorflow-text tfx==1.12 tensorflow-datasets transformers keras-nlp markupsafe

In [1]:
%load_ext autoreload
%autoreload 2

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import jax
import jax.numpy as jnp
import flax
import transformers
import optax
from flax.training import train_state
import tensorflow as tf
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_datasets as tfds

import os
import tempfile
from transformers import T5TokenizerFast
import tensorflow_text as tf_text

import constants


# import tfx components
from tfx.components import CsvExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer

import preprocessing
from transformers import TFT5ForConditionalGeneration



2023-03-20 13:41:42.528823: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib::/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64/:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64/:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64/:/usr/local/cuda/lib64:/usr/local/cuda/ex

# Objective
We'll be building a data engine for a generative machine learning model. The objective is to create a data driven approach to improve the machine learning model and along the way learn about MLOps, Feature Engineering, Model Serving and Model Analysis. In the end, you should be able to use this model to write more information dense sentences and improve the way you speak and write. Let's see how to achieve that.

## The dataset
We are going to need some data to train on as an initial starting point. After exploring several datasets, I settled on using the [Asset Dataset](https://www.tensorflow.org/datasets/catalog/asset). 

In [4]:
# This is the root directory for your TFX pip package installation.
ROOT_DIR = "/home/azureuser/"


# remove all previous artifacts
!rm -rf {os.path.join(ROOT_DIR, "pipelines/artifacts")}

ARTIFACT_DIR = os.path.join(ROOT_DIR, "pipelines/artifacts")
MODEL_SERVING_DIR = os.path.join(ROOT_DIR, "serving_model/generative_dense_writing")
DATA_DIR = os.path.join(ROOT_DIR, "datasets/curated/"); 
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
DATASET_NAME = "asset"


we'll create a csv out of it. But before we do that, we'll need to add a unique id to each row. The reason we do this is that if we split the dataset into training and test dataset, we can use the ID for splitting. Otherwise, we might introduce evaluation bias

In [5]:
# creata a unique id per row
import uuid
ds, ds_info = tfds.load(DATASET_NAME, with_info=True)
# concatenate validation and test datasets
df = tfds.as_dataframe(ds["validation"], ds_info=ds_info)

df["uuid"] = df.apply(lambda x: str(uuid.uuid4()), axis = 1)

# decode original and simplified text
df["text"] = df["original"].apply(lambda x: x.decode("utf-8"))
df["simplifications"] = df["simplifications"].apply(lambda x: [s.decode("utf-8") for s in x])

# simplifications has a list of strings, i want to have a new row per string in the list
df = df.explode("simplifications")

df.to_csv(DATA_DIR + "curated.csv", index=False)
df.head()

2023-03-20 13:43:10.361481: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


,original,simplifications,uuid,text
0,"b'Her due release date, with remissions, is cu...",Her current release date is 12 April 2024.,9b6459dc-af47-4db5-9e5c-1473298ce21f,"Her due release date, with remissions, is curr..."
0,"b'Her due release date, with remissions, is cu...",Her release date is 12 April 2024.,9b6459dc-af47-4db5-9e5c-1473298ce21f,"Her due release date, with remissions, is curr..."
0,"b'Her due release date, with remissions, is cu...","April 12, 2024 is when she is due to be released.",9b6459dc-af47-4db5-9e5c-1473298ce21f,"Her due release date, with remissions, is curr..."
0,"b'Her due release date, with remissions, is cu...","Her release date is April 12th, 2024.",9b6459dc-af47-4db5-9e5c-1473298ce21f,"Her due release date, with remissions, is curr..."
0,"b'Her due release date, with remissions, is cu...",Her planned release date is 12 April 2024.,9b6459dc-af47-4db5-9e5c-1473298ce21f,"Her due release date, with remissions, is curr..."


In [6]:
from ml_metadata.proto import metadata_store_pb2
PIPELINE_NAME = "generative_dense_writing"

In [7]:
context = InteractiveContext(PIPELINE_NAME, pipeline_root=ARTIFACT_DIR)

In [8]:

example_gen = tfx.components.CsvExampleGen(input_base=DATA_DIR)
context.run(example_gen, enable_cache=True)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [9]:
# print 5 examples from the TFRecord file
import tensorflow as tf

examples_root_dir = example_gen.outputs['examples'].get()[0] 
# get train and eval files
examples = tf.io.gfile.glob(os.path.join(examples_root_dir.uri, 'Split-train/*'))
# read the first example
raw_record = tf.data.TFRecordDataset(examples, compression_type="GZIP").take(1)

# deserialize the example
for raw_record in raw_record:
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)


features {
  feature {
    key: "original"
    value {
      bytes_list {
        value: "b\'Her due release date, with remissions, is currently 12 April 2024.\'"
      }
    }
  }
  feature {
    key: "simplifications"
    value {
      bytes_list {
        value: "Her release date is 12 April 2024."
      }
    }
  }
  feature {
    key: "text"
    value {
      bytes_list {
        value: "Her due release date, with remissions, is currently 12 April 2024."
      }
    }
  }
  feature {
    key: "uuid"
    value {
      bytes_list {
        value: "9b6459dc-af47-4db5-9e5c-1473298ce21f"
      }
    }
  }
}



In [10]:
# build the pipeline
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen, enable_cache=True)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [11]:
# display schema using tfdv
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'original',BYTES,required,,-
'simplifications',BYTES,required,,-
'text',BYTES,required,,-
'uuid',BYTES,required,,-


In [12]:
# Build the schema validator
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

Bad pipe message: %s [b'\x91\x0f\xa7g\xc4\x17k\xbe\xf9A\xceT!\xd3Z\x83\x9f\x1d \xbef~\xc8\xfc\xf5\xda\xb2\x04\x05\xf6\xbb\xc3\xc8F\xfe$\x0c\xb4\x80|\xf7\xd31\x7f\x15u|i\xa5\xd99\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d']
Bad pipe message: %s [b"\xcb\xde\xa89\xf7\x82\xec\xce\xce\x05\xaf\x12\xa0\xd3:y/\x8d\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00

In [15]:
tokenizer = preprocessing.create_tf_text_tokenizer()


In [17]:
# import preprocessing
preprocessing.preprocessing_fn({"text": tf.constant([["hello world"], ["another"]]), "simplifications": tf.constant([["hello world"], ["another"]])})

{'input_ids': <tf.Tensor: shape=(2, 512), dtype=int64, numpy=
 array([[21820,   296,     0, ...,     0,     0,     0],
        [  430,     0,     0, ...,     0,     0,     0]])>,
 'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int64, numpy=
 array([[1, 1, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]])>}

In [18]:
_generative_transform_module_file = 'preprocessing.py'
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_generative_transform_module_file))
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying gen_transform.py -> build/lib
copying model.py -> build/lib
copying preprocessing.py -> build/lib
copying task.py -> build/lib
installing to /tmp/tmpc5s9di6g
running install
running install_lib
copying build/lib/constants.py -> /tmp/tmpc5s9di6g
copying build/lib/preprocessing.py -> /tmp/tmpc5s9di6g
copying build/lib/task.py -> /tmp/tmpc5s9di6g
copying build/lib/model.py -> /tmp/tmpc5s9di6g
copying build/lib/gen_transform.py -> /tmp/tmpc5s9di6g
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.tx

Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: /home/azureuser/pipelines/artifacts/Transform/transform_graph/5/.temp_path/tftransform_tmp/852c11ced8c24bb9880f6f6cccdf2f51/assets


INFO:tensorflow:Assets written to: /home/azureuser/pipelines/artifacts/Transform/transform_graph/5/.temp_path/tftransform_tmp/852c11ced8c24bb9880f6f6cccdf2f51/assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [19]:
import tensorflow_transform as tft
# output_dir = "/home/azureuser/pipelines/artifacts/Transform/transform_graph/123"
output_dir = transform.outputs["transform_graph"]._artifacts[0].uri
tf_transform_output = tft.TFTransformOutput(output_dir)
tf_transform_output.transformed_feature_spec()


{'attention_mask': FixedLenFeature(shape=[512], dtype=tf.int64, default_value=None),
 'input_ids': FixedLenFeature(shape=[512], dtype=tf.int64, default_value=None)}

In [80]:
TOKENIZER = preprocessing.create_tf_text_tokenizer(constants.PRETRAINED_MODEL_FAMILY)


In [20]:

tf_transform_output.raw_feature_spec()

{'original': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None),
 'simplifications': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None),
 'text': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None),
 'uuid': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None)}

In [21]:

import model
t5_llm = model.build_model()
t5_llm.compile()

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [17]:

# t5_llm.tft_layer_inference = tf_transform_output.transform_features_layer()
# transformed_features = t5_llm.tft_layer_inference({"text": tf.constant([["hello world"]])})
# outputs = t5_llm.generate(
#     input_ids=transformed_features["input_ids"],
#     attention_mask=transformed_features["attention_mask"],
#     max_new_tokens=constants.DECODER_MAX_LEN,
#     return_dict_in_generate=True,
# )



In [18]:
# sentences = tf.reshape(t5_llm.tokenizer.detokenize(outputs["sequences"]), (-1, 1))
# sentences

In [22]:

metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name="accuracy")]
t5_llm.compile(metrics=metrics)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [29]:
t5_llm = TFT5ForConditionalGeneration.from_pretrained("t5-small")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [38]:
t5_llm.compile(optimizer='adam')
t5_llm.fit()

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>

In [28]:

import model
MODEL_SERVING_DIR = os.path.join(ROOT_DIR, "serving_model/generative_dense_writing")



def _get_serve_features_signature(t5_llm, tf_transform_output):
    """Returns a function that parses a raw inputs and applies TFT."""

    t5_llm.tft_layer_inference = tf_transform_output.transform_features_layer()

    @tf.function(
        input_signature=(tf.TensorSpec((None, ), tf.string, name="text"),)
    )
    def serving(text):

        text = tf.reshape(text, shape=[-1, 1])

        # convert text to a sparse tensor
        # text = tf.sparse.from_dense(text)

        transformed_features = t5_llm.tft_layer_inference({"text": text})

        outputs = t5_llm.generate(
            input_ids=transformed_features["input_ids"],
            attention_mask=transformed_features["attention_mask"],
            max_new_tokens=constants.DECODER_MAX_LEN,
            return_dict_in_generate=True,
        )

        return transformed_features


        # return outputs

        # sentences = model.tokenizer.detokenize(outputs["sequences"])
        
        # return {"outputs" : sentences}

    return serving

signatures = {
    'serving_default':
        _get_serve_features_signature(t5_llm,
                            tf_transform_output)
}
t5_llm.save(MODEL_SERVING_DIR, save_format='tf', signatures=signatures)

AssertionError: in user code:

    File "/tmp/ipykernel_1696268/3752083138.py", line 23, in serving  *
        outputs = t5_llm.generate(
    File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/generation_tf_utils.py", line 748, in generate  *
        assert (

    AssertionError: expected encoder_outputs[0] to have 1st dimension bs=Tensor("strided_slice:0", shape=(), dtype=int32), got None 


In [39]:
loaded_model = tf.saved_model.load(MODEL_SERVING_DIR)

In [42]:
prediction_wit_loaded_model = loaded_model.signatures['serving_default'](
    text=tf.constant(["hello world", "another"])
)
prediction_wit_loaded_model

{'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int64, numpy=
 array([[1, 1, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]])>,
 'input_ids': <tf.Tensor: shape=(2, 512), dtype=int64, numpy=
 array([[21820,   296,     0, ...,     0,     0,     0],
        [  430,     0,     0, ...,     0,     0,     0]])>}

In [43]:
t5_llm.generate(
    input_ids=prediction_wit_loaded_model["input_ids"],
    attention_mask=prediction_wit_loaded_model["attention_mask"],
    max_new_tokens=constants.DECODER_MAX_LEN,
    return_dict_in_generate=True,
)["sequences"]

<tf.Tensor: shape=(2, 20), dtype=int32, numpy=
array([[    0, 21820, 21820, 21820, 21820, 21820, 21820, 21820, 21820,
        21820, 21820, 21820, 21820, 21820, 21820, 21820, 21820, 21820,
        21820, 21820],
       [    0,   430,   430,   430,   430,   430,   430,   430,   430,
          430,   430,   430,   430,   430,   430,   430,   430,   430,
          430,   430]], dtype=int32)>

In [17]:
def _make_training_input_fn(tf_transform_output, train_file_pattern,
                            batch_size):
  """An input function reading from transformed data, converting to model input.

  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    transformed_examples: Base filename of examples.
    batch_size: Batch size.

  Returns:
    The input data for training or eval, in the form of k.
  """
  def input_fn():
    return tf.data.experimental.make_batched_features_dataset(
        file_pattern=train_file_pattern,
        batch_size=batch_size,
        features=tf_transform_output.transformed_feature_spec(),
        reader=tf.data.TFRecordDataset,
        label_key="simplifications",
        shuffle=True)

  return input_fn

In [18]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

def parse_single_example(x, features_spec=tf_transform_output.transformed_feature_spec()):
    return tf.io.parse_single_example(x, features_spec)

ds = dataset.map(parse_single_example)
ds = ds.batch(4)
ds = ds.prefetch(1)
ds = ds.skip(100)
ds = ds.take(1)
for x in ds:
    print(x)

{'attention_mask': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'decoder_attention_mask': <tf.Tensor: shape=(4, 64), dtype=int64, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
    

In [32]:


TOKENIZER = preprocessing.load_pretrained_tokenizer_as_tf_text_tokenizer()
MODEL_FAMILY = "t5-large"
model = TFT5ForConditionalGeneration.from_pretrained(MODEL_FAMILY)

Filenames in tmp_dir: ['spiece.model']


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [33]:

outputs = model.generate(
    x["input_ids"], 
    attention_mask=x["attention_mask"], 
    max_new_tokens=constants.DECODER_MAX_LEN,
    return_dict_in_generate=True
)

In [34]:
decoded_output = TOKENIZER.detokenize(outputs["sequences"])
print(decoded_output)

tf.Tensor(
[b'<extra_id_99>, Canada, Canada, Canada, Canada, Canada, Canada, Canada, Canada, Canada'
 b'<extra_id_99>, Canada, Canada, Canada, Canada, Canada, Canada, Canada, Canada, Canada'
 b'<extra_id_99>, Canada, Canada, Canada, Canada, Canada, Canada, Canada, Canada, Canada'
 b'<extra_id_99>, Canada, Canada, Canada, Canada, Canada, Canada, Canada, Canada, Canada'], shape=(4,), dtype=string)


In [37]:
text =      "translate english to german: My name is Sarah and I live in London"
features = preprocessing.preprocessing_fn({"text": tf.constant([[text]]), "simplifications": tf.constant([[""]])})

outputs = model.generate(
    features["input_ids"], 
    attention_mask=features["attention_mask"], 
    max_new_tokens=constants.DECODER_MAX_LEN,
    return_dict_in_generate=True
)

tokenizer.detokenize(outputs["sequences"])

Filenames in tmp_dir: ['spiece.model']


<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Mein Name ist Sarah und ich lebe in London.'], dtype=object)>

In [59]:

tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = TFT5ForConditionalGeneration.from_pretrained("t5-large")

# inference
inputs = tokenizer(
text, return_tensors="tf"
).input_ids  # Batch size 1
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Mein Name ist Sarah und ich lebe in London


In [13]:
MODEL_FAMILY = "t5-small"
text = "translate english to german: My name is Sarah and I live in London",
text = "know it for hosting a maximum-security prison holding especially dangerous Mafia criminals".lower()
tokenizer = preprocessing.load_pretrained_tokenizer_as_tf_text_tokenizer(MODEL_FAMILY)
text = tf.reshape(text, [-1, 1])
features = preprocessing.preprocessing_fn({"text": text, "simplifications": text})

Filenames in tmp_dir: ['spiece.model']


2023-03-15 15:23:34.787403: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Filenames in tmp_dir: ['spiece.model']


In [14]:
tokenizer.vocab_size()

<tf.Tensor: shape=(), dtype=int32, numpy=32100>

In [14]:
from transformers import T5TokenizerFast
T5TokenizerFast.from_pretrained("t5-small")

PreTrainedTokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<ex

In [19]:
import requests 
import tensorflow_text
direct_tok = tensorflow_text.SentencepieceTokenizer(requests.get("https://huggingface.co/t5-small/resolve/main/spiece.model").content)

In [20]:
direct_tok.vocab_size()

<tf.Tensor: shape=(), dtype=int32, numpy=32000>

In [23]:
!pip install sentencepiece
import sentencepiece.sentencepiece_model_pb2 as model

In [16]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("t5-small")

In [15]:

model = TFT5ForConditionalGeneration.from_pretrained(MODEL_FAMILY)
outputs = model.generate(
    features["input_ids"], 
    attention_mask=features["attention_mask"], 
    max_new_tokens=constants.DECODER_MAX_LEN,
    return_dict_in_generate=True
)
decoded_output = tokenizer.detokenize(outputs["sequences"][0])
print(decoded_output)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


tf.Tensor(b'<extra_id_99> for hosting maximum-security prison holding especially dangerous mafia criminals . ma', shape=(), dtype=string)


In [9]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
model = TFT5ForConditionalGeneration.from_pretrained(MODEL_FAMILY)
outputs = model.generate(
    features["input_ids"], 
    attention_mask=features["attention_mask"], 
    max_new_tokens=constants.DECODER_MAX_LEN,
    return_dict_in_generate=True
)
decoded_output = tokenizer.detokenize(outputs["sequences"][0])
print(decoded_output)


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
2023-03-15 08:22:45.094690: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at sentencepiece_kernels.cc:615 : OUT_OF_RANGE: Invalid id: 32099


OutOfRangeError: {{function_node __wrapped__SentencepieceDetokenizeOp_device_/job:localhost/replica:0/task:0/device:CPU:0}} Invalid id: 32099 [Op:SentencepieceDetokenizeOp]